In [11]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data for CNN
x_train_cnn = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test_cnn = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255
y_train_cnn = to_categorical(y_train)
y_test_cnn = to_categorical(y_test)

# Preprocess the data for Random Forest
x_train_rf = x_train.reshape((x_train.shape[0], 28 * 28))
x_test_rf = x_test.reshape((x_test.shape[0], 28 * 28))

# Train the CNN model with improved architecture and regularization
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.5))  # Increased dropout for regularization
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(10, activation='softmax'))
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Apply early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

cnn_model.fit(x_train_cnn, y_train_cnn, epochs=10, batch_size=64, validation_data=(x_test_cnn, y_test_cnn), callbacks=[early_stopping])

# Extract CNN features
cnn_features_train = cnn_model.predict(x_train_cnn)
cnn_features_test = cnn_model.predict(x_test_cnn)

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=500, min_samples_leaf=2)  # Adjusted hyperparameters
rf_model.fit(x_train_rf, y_train)

# Extract Random Forest features
rf_features_train = rf_model.predict_proba(x_train_rf)
rf_features_test = rf_model.predict_proba(x_test_rf)

# Combine features
combined_features_train = np.concatenate((cnn_features_train, rf_features_train), axis=1)
combined_features_test = np.concatenate((cnn_features_test, rf_features_test), axis=1)

# Train a classifier on combined features
classifier = Sequential()
classifier.add(Dense(256, activation='relu', input_shape=(combined_features_train.shape[1],)))
classifier.add(Dropout(0.5))
classifier.add(Dense(10, activation='softmax'))
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(combined_features_train, y_train_cnn, epochs=10, batch_size=64, validation_data=(combined_features_test, y_test_cnn), callbacks=[early_stopping])

# Evaluate the ensemble model
_, ensemble_accuracy = classifier.evaluate(combined_features_test, y_test_cnn)
print('Ensemble Accuracy: %.2f%%' % (ensemble_accuracy * 100))


Epoch 1/10
938/938 [==============================] - 13s 14ms/step - loss: 0.2748 - accuracy: 0.9143 - val_loss: 0.0593 - val_accuracy: 0.9821
Epoch 2/10
938/938 [==============================] - 13s 13ms/step - loss: 0.0900 - accuracy: 0.9732 - val_loss: 0.0369 - val_accuracy: 0.9875
Epoch 3/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0666 - accuracy: 0.9804 - val_loss: 0.0344 - val_accuracy: 0.9890
Epoch 4/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0546 - accuracy: 0.9839 - val_loss: 0.0252 - val_accuracy: 0.9910
Epoch 5/10
938/938 [==============================] - 13s 13ms/step - loss: 0.0454 - accuracy: 0.9866 - val_loss: 0.0279 - val_accuracy: 0.9910
Epoch 6/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0393 - accuracy: 0.9882 - val_loss: 0.0234 - val_accuracy: 0.9931
Epoch 7/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0337 - accuracy: 0.9895 - val_loss: 0.0284 - val_accuracy:

In [17]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data for CNN
x_train_cnn = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test_cnn = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255
y_train_cnn = to_categorical(y_train)
y_test_cnn = to_categorical(y_test)

# Preprocess the data for Random Forest
x_train_rf = x_train.reshape((x_train.shape[0], 28 * 28))
x_test_rf = x_test.reshape((x_test.shape[0], 28 * 28))

# Initialize list to store accuracy for each run
ensemble_accuracies = []

# Perform 5 runs
for run in range(5):
    print(f"Run {run + 1}:")

    # Train the CNN model with improved architecture and regularization
    cnn_model = Sequential()
    cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    cnn_model.add(MaxPooling2D((2, 2)))
    cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
    cnn_model.add(MaxPooling2D((2, 2)))
    cnn_model.add(Flatten())
    cnn_model.add(Dense(128, activation='relu'))
    cnn_model.add(Dropout(0.5))  # Increased dropout for regularization
    cnn_model.add(Dense(64, activation='relu'))
    cnn_model.add(Dense(10, activation='softmax'))
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Apply early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train CNN model for 10 epochs
    cnn_model.fit(x_train_cnn, y_train_cnn, epochs=10, batch_size=64, validation_data=(x_test_cnn, y_test_cnn), callbacks=[early_stopping])

    # Extract CNN features
    cnn_features_train = cnn_model.predict(x_train_cnn)
    cnn_features_test = cnn_model.predict(x_test_cnn)

    # Train the Random Forest model
    rf_model = RandomForestClassifier(n_estimators=500, min_samples_leaf=2)  # Adjusted hyperparameters
    rf_model.fit(x_train_rf, y_train)

    # Extract Random Forest features
    rf_features_train = rf_model.predict_proba(x_train_rf)
    rf_features_test = rf_model.predict_proba(x_test_rf)

    # Combine features
    combined_features_train = np.concatenate((cnn_features_train, rf_features_train), axis=1)
    combined_features_test = np.concatenate((cnn_features_test, rf_features_test), axis=1)

    # Train a classifier on combined features
    classifier = Sequential()
    classifier.add(Dense(256, activation='relu', input_shape=(combined_features_train.shape[1],)))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(10, activation='softmax'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the classifier for 10 epochs
    classifier.fit(combined_features_train, y_train_cnn, epochs=10, batch_size=64, validation_data=(combined_features_test, y_test_cnn), callbacks=[early_stopping])

    # Evaluate the ensemble model
    _, ensemble_accuracy = classifier.evaluate(combined_features_test, y_test_cnn)
    ensemble_accuracies.append(ensemble_accuracy)
    print('Ensemble Accuracy: %.2f%%' % (ensemble_accuracy * 100))

# Calculate and print the average accuracy over 5 runs
avg_accuracy = np.mean(ensemble_accuracies)
print('Average Ensemble Accuracy: %.2f%%' % (avg_accuracy * 100))


Run 1:
Epoch 1/10
938/938 [==============================] - 14s 14ms/step - loss: 0.2466 - accuracy: 0.9229 - val_loss: 0.0507 - val_accuracy: 0.9842
Epoch 2/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0816 - accuracy: 0.9765 - val_loss: 0.0324 - val_accuracy: 0.9881
Epoch 3/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0593 - accuracy: 0.9824 - val_loss: 0.0315 - val_accuracy: 0.9900
Epoch 4/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0506 - accuracy: 0.9853 - val_loss: 0.0275 - val_accuracy: 0.9906
Epoch 5/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0438 - accuracy: 0.9868 - val_loss: 0.0236 - val_accuracy: 0.9920
Epoch 6/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0347 - accuracy: 0.9898 - val_loss: 0.0289 - val_accuracy: 0.9912
Epoch 7/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0323 - accuracy: 0.9900 - val_loss: 0.0260 - val_ac